In [ ]:
import pandas as pd
import os 
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set_theme()


In [ ]:
form_ = pd.read_excel('Form_ answers.xlsx', sheet_name=None)
form = pd.read_excel('Form Answers.xlsx', sheet_name=None)

In [ ]:
subjects = pd.read_excel('Experiment.xlsx', header=None)
subjects.loc[subjects[0] == 'aniabzinkowska@gmail.com', 0] = 's213027@student.dtu.dk'
subjects.loc[subjects[0] == 'toobias.hartel@gmail.com', 0] ='toobias.hartel@googlemail.com'
subjects = subjects.iloc[:-1, :]
subjects.columns = ['Email', 'SubjectID', 'Form']
subjects['SubjectID'] = subjects['SubjectID'].astype(int)
subjects['SubjectID'] = subjects['SubjectID'].astype(str)
subjects['Email'] = subjects['Email'].str.lower()
subjects['Email'] = subjects['Email'].str.strip()

subjects

In [ ]:
#datasets for activities

first_date = datetime.date.fromisoformat('2022-04-25')
form__activities = pd.DataFrame(columns=['Date', 'Stress Score','Stress Level', 'Category', 'Activity', 'Email'])
form_activites = pd.DataFrame(columns=['Date', 'Stress Score','Stress Level', 'Category', 'Activity', 'Email'])

dict_scores_ = {'Not at all': 0, 'Somewhat': 1, 'Moderately': 2, 'Very much so': 3}
dict_scores = {'Calm': {'Not at all': 3, 'Somewhat': 2, 'Moderately': 1, 'Very much so': 0}, 
                'Tense': {'Not at all': 0, 'Somewhat': 1, 'Moderately': 2, 'Very much so': 3},
                'Upset': {'Not at all': 0, 'Somewhat': 1, 'Moderately': 2, 'Very much so': 3},
                'Relaxed': {'Not at all': 3, 'Somewhat': 2, 'Moderately': 1, 'Very much so': 0},
                'Content': {'Not at all': 3, 'Somewhat': 2, 'Moderately': 1, 'Very much so': 0},
                'Worried': {'Not at all': 0, 'Somewhat': 1, 'Moderately': 2, 'Very much so': 3}}

stress_map = {1: 'No Stress', 2: 'Low', 3: 'Moderate', 4: 'High'}

#obtain dataframe for form_ responses
for i in range(1,8):
    date = first_date + datetime.timedelta(-1 + i)
    sheet_name = 'Respuestas de formulario {}'.format(i)
    df = form_[sheet_name].dropna(subset='Marca temporal')
    for i, row in df.iterrows():
        email = row[1].strip().lower()
        stress_score = dict_scores_[row[2]] + 1
        for category in ['Social', 'Work', 'Studies', 'Health', 'Hobbies']:
            activities = row[category]
            if not pd.isnull(activities):
                for activity in activities.split(','):
                    form__activities.loc[len(form__activities)] = [date, stress_score, stress_map[stress_score], category, activity.strip(), email]
        
#obtain dataframe for form responses
for i in range(1,8):
    date = first_date + datetime.timedelta(-1 + i)
    sheet_name = 'Respuestas de formulario {}'.format(i)
    df = form[sheet_name].dropna(subset='Marca temporal')
    for i, row in df.iterrows():
        email = row[1].strip().lower()
        
        stress_score = dict_scores['Calm'][row[2]] + dict_scores['Tense'][row[3]] + dict_scores['Upset'][row[4]]
        + dict_scores['Relaxed'][row[5]] + dict_scores['Worried'][row[6]]
        
        stress_score  = stress_score * 100 / 9
        stress_score = stress_score * 3 / 100 + 1

        for category in ['Social', 'Work', 'Studies', 'Health', 'Hobbies']:
            activities = row[category]
            if not pd.isnull(activities):
                for activity in activities.split(','):
                    form_activites.loc[len(form_activites)] = [date, stress_score, stress_map[int(round(stress_score,0))], category, activity.strip(), email]


for i, row in subjects[subjects['Form'] == '5-point Likert Scale'].iterrows():
    form__activities[form__activities['Email'] == row[0]].to_csv('Results/{}_subject_{}_activities.csv'.format(0, row[1]))

for i, row in subjects[subjects['Form'] == 'Short STAI'].iterrows():
    form_activites[form_activites['Email'] == row[0]].to_csv('Results/{}_subject_{}_activities.csv'.format(1, row[1]))

In [ ]:
#datasets for stress level

first_date = datetime.date.fromisoformat('2022-04-25')

dict_scores_ = {'Not at all': 0, 'Somewhat': 1, 'Moderately': 2, 'Very much so': 3}
dict_scores = {'Calm': {'Not at all': 3, 'Somewhat': 2, 'Moderately': 1, 'Very much so': 0}, 
                'Tense': {'Not at all': 0, 'Somewhat': 1, 'Moderately': 2, 'Very much so': 3},
                'Upset': {'Not at all': 0, 'Somewhat': 1, 'Moderately': 2, 'Very much so': 3},
                'Relaxed': {'Not at all': 3, 'Somewhat': 2, 'Moderately': 1, 'Very much so': 0},
                'Content': {'Not at all': 3, 'Somewhat': 2, 'Moderately': 1, 'Very much so': 0},
                'Worried': {'Not at all': 0, 'Somewhat': 1, 'Moderately': 2, 'Very much so': 3}}

#relevant dfs
form__responses = pd.DataFrame(columns=['Date', 'Stress Score', 'Stress Level', 'Social', 'Work', 'Studies', 'Health', 'Hobbies', 'Email'])
form_responses = pd.DataFrame(columns=['Date', 'Stress Score', 'Stress Level', 'Social', 'Work', 'Studies', 'Health', 'Hobbies', 'Email'])

for i in range(1,8):
    date = first_date + datetime.timedelta(-1 + i)
    sheet_name = 'Respuestas de formulario {}'.format(i)
    df = form_[sheet_name].dropna(subset='Marca temporal')
    for i, row in df.iterrows():
        email = row[1].strip().lower()
        row.fillna('No activities registered', inplace=True)
        stress_score = dict_scores_[row[2]] + 1  
        form__responses.loc[len(form__responses)] = [date, stress_score, stress_map[stress_score], row['Social'], row['Work'], row['Studies'], row['Health'], row['Hobbies'], email]

#obtain dataframe for form responses
for i in range(1,8):
    date = first_date + datetime.timedelta(-1 + i)
    sheet_name = 'Respuestas de formulario {}'.format(i)
    df = form[sheet_name].dropna(subset='Marca temporal')
    for i, row in df.iterrows():
        email = row[1].strip().lower()
        row.fillna('No activities registered', inplace=True)

        stress_score = dict_scores['Calm'][row[2]] + dict_scores['Tense'][row[3]] + dict_scores['Upset'][row[4]]
        + dict_scores['Relaxed'][row[5]] + dict_scores['Worried'][row[6]]
        
        stress_score  = stress_score * 100 / 9
        stress_score = stress_score * 3 / 100 + 1

        form_responses.loc[len(form_responses)] = [date, stress_score, stress_map[int(round(stress_score,0))], row['Social'], row['Work'], row['Studies'], row['Health'], row['Hobbies'], email]
        
for i, row in subjects[subjects['Form'] == '5-point Likert Scale'].iterrows():
    form__responses[form__responses['Email'] == row[0]].to_csv('Results/{}_subject_{}.csv'.format(0, row[1]))

for i, row in subjects[subjects['Form'] == 'Short STAI'].iterrows():
    form_responses[form_responses['Email'] == row[0]].to_csv('Results/{}_subject_{}.csv'.format(1, row[1]))